In [ ]:
# 移动文件
# import shutil
# shutil.move("data/data101045/words.vector.gz","./")

'./words.vector.gz'

1. 基础的文本数据增强（EDA）[同义词替换!]
2. 闭包数据增强
3. 无监督数据增强（UDA）
4. 对偶数据增强

# 数据增强 （Data Augmentation）

数据增强：通过有限数据来产生更多的等价数据，人工扩展训练集的技术

In [ ]:
! pip install -U synonyms

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 10.7MB 17.7MB/s eta 0:00:01
  Created wheel for synonyms: filename=synonyms-3.16.0-cp37-none-any.whl size=10832785 sha256=11e02e419dbacde24a3e357f2e42ec8f506a9d90b1acb412d3f663f0eb12f03d
  Stored in directory: /home/aistudio/.cache/pip/wheels/cf/62/50/96a495f5ed067aaf5425e9debf31980c31ecb1bbe27f37f862
Successfully built synonyms


In [ ]:
! cp 'words.vector.gz' '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/synonyms/data'

In [ ]:
import synonyms

[jieba] default dict file path ../data/vocab.txt
[jieba] default dict file path ../data/vocab.txt
[jieba] load default dict ../data/vocab.txt ...
[jieba] load default dict ../data/vocab.txt ...
>> Synonyms load wordseg dict [/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/synonyms/data/vocab.txt] ... 
>> Synonyms on loading stopwords [/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/synonyms/data/words.vector.gz] ...


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
synonyms.nearby('爸爸')

(['爸爸', '妈妈', '爷爷', '奶奶', '老婆', '姐姐', '爸妈', '老公', '老爸', '外婆'],
 [1.0,
  0.9197904,
  0.8369896,
  0.83030784,
  0.80404544,
  0.79813045,
  0.79792285,
  0.7714373,
  0.77116084,
  0.75844836])

## EDA（Easy Data Augmentation）

![UDA1](https://img-blog.csdnimg.cn/c5ffcca4482c4c42beb6f1215e37657c.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### EDA：用于提高文本分类任务性能的简单数据增强技术。 EDA 由四个简单但功能强大的操作组成：同义词替换、随机插入、随机交换和随机删除。

![EDA2](https://img-blog.csdnimg.cn/3fc3cec8b4d24c30939a92c1fac93b35.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 之前的工作已经提出了一些 NLP 中数据增强的技术，回译通过将句子翻译成法语然后再翻译成英语来生成新数据。

![EDA3](https://img-blog.csdnimg.cn/50c22b4212714b509ce053ff921d6bdd.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 对于训练集中的给定句子，随机选择并执行以下操作之一：
* 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
* 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。
* 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
* 随机删除（RD）：以概率 p 随机删除句子中的每个单词。

In [ ]:
stop_words = {word.strip() for word in open('baidu_stopwords.txt', 'r', encoding='utf8').readlines()}

In [ ]:
# # 同义词替换（SR）：从句子中随机选择 n 个不是停用词的词。 用随机选择的同义词之一替换这些单词中的每一个。
# import random
# def get_synonyms(word):
#     # (['mama'],['0.9'])
#     sys = set(synonyms.nearby(word)[0])
#     if word in sys:
#         sys.remove(word)
#     return list(sys)

# # 给力  ([],[])
# def synonym_replacement(words, n):
#     new_words = words.copy()
#     # 不在停用词表
#     random_word_list = list(set([word for word in words if word not in stop_words]))
#     random.shuffle(random_word_list)
#     num_replaced = 0
#     for random_word in random_word_list:
#         synonyms = get_synonyms(random_word)
#         if len(synonyms) >=1 :
#             synonym = random.choice(list(synonyms))
#             new_words = [synonym if word == random_word else word for word in new_words]
#             num_replaced += 1
#         if num_replaced >= n:
#             break
    
#     # new_words = [ ]
#     sentence = ' '.join(new_words)
#     new_words = sentence.split(' ')
#     return new_words

In [ ]:
# 存在一种情况  actor -> film star
sentence = ['in', 'actor']
new_words = ['in', 'film', 'star']

In [ ]:
# 随机删除（RD）：以概率 p 随机删除句子中的每个单词。
def random_deletion(words, p):
    if len(words) == 1:
        return words
    
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r>p:
            new_words.append(word)
    
    if len(new_words) == 0:
        random_int = random.randint(0, len(words)-1)
        return [words[random_int]]
    
    return new_words

In [ ]:
# 随机交换（RS）：随机选择句子中的两个单词并交换它们的位置。 这样做n次。
def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1

    counter = 0
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)
    return new_words

In [ ]:
# 随机插入 (RI)：在句子中随机找到一个词，并找出其同义词，且该同义词不是停用词。 将该同义词插入句子中的随机位置。 这样做n次。

def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        # 在句子中随机找到一个词
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return 
    
    random_synonym = synonyms[0]
    # 取出要插入的位置
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

def random_insert(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    return new_words

In [ ]:
from random import shuffle
# num_aug 增加了几条数据
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    words = synonyms.seg(sentence)[0]
    num_words = len(words)
    augmented_sentence = []
    # 3
    num_new_per_technique = int(num_aug/4) + 1

    n_sr = max(1, int(alpha_sr*num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))
    
    # 同义词替换
    # for _ in range(num_new_per_technique):
    #     a_words = synonym_replacement(words, n_sr)
    #     # a_words []
    #     print(a_words)
    #     augmented_sentence.append(' '.join(a_words))
    
    # 随机插入
    for _ in range(num_new_per_technique):
        a_words = random_insert(words, n_ri)
        augmented_sentence.append(' '.join(a_words))

    # 随机交换
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentence.append(' '.join(a_words))
    
    # 随机删除
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentence.append(' '.join(a_words))
    
    shuffle(augmented_sentence)

    if num_aug>=1:
        augmented_sentence = augmented_sentence[:num_aug]
    
    else: #num_aug<1
        keep_prob = num_aug/len(augmented_sentence)
        # random_delete
        augmented_sentence = [s for s in augmented_sentence if random.uniform(0,1)<keep_prob]
    
    return augmented_sentence

In [ ]:
words = synonyms.seg('目前华为部分型号的手机产品出现货少的现象')
print(words)

(['目前', '华为', '部分', '型号', '的', '手机', '产品', '出现', '货', '少', '的', '现象'], ['t', 'nr', 'n', 'n', 'uj', 'n', 'n', 'v', 'n', 'a', 'uj', 'n'])


In [ ]:
eda('9月15日以来，台积电、高通、三星等华为的重要合作伙伴，只要没有美国的相关许可证，都无法供应芯片给华为，而中芯国际等国产芯片企业，也因采用美国技术，而无法供货给华为。目前华为部分型号的手机产品出现货少的现象，若该形势持续下去，华为手机业务将遭受重创。')

['若 月 15 日 以来 的 台积电 、 ， 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 ， 相关 许可证 ， 都 无法 供应 芯片 的 华为 ， 而 中芯国际 等 将 手机 企业 ， 也 因 采用 美国 技术 高通 而 无法 供货 给 华为 。 目前 华为 部分 型号 给 手机 产品 出现 货 少 的 现象 ， 9 该 形势 持续 下去 ， 华为 芯片 业务 国产 遭受 重创 。',
 '9 采用 15 日 以来 ， 台积电 ， 高通 、 三星 等 华为 的 将 合作伙伴 ， 只要 没有 美国 若 相关 许可证 ， 都 无法 供应 芯片 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 月 美国 技术 ， 而 无法 供货 给 华为 。 华为 华为 部分 型号 的 手机 产品 出现 货 少 的 现象 ， 重创 该 形势 持续 下去 、 目前 手机 的 重要 遭受 业务 。',
 '9 月 15 日 ， 台积电 、 高通 、 三星 等 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 给 华为 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 供货 给 华为 。 目前 华为 部分 型号 的 手机 产品 出现 货 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 以来 ， 台积电 、 高通 、 三星 相关人员 等 EMC 中央处理器 反倒 华为 的 重要 合作伙伴 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 初期 给 华为 ， 而 中芯国际 等 国产 芯片 企业 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 猪仔 。 目前 华为 部分 型号 的 贵 手机 产品 出现 货 少 的 现象 ， 若 该 形势 持续 下去 ， 华为 手机 业务 将 遭受 重创 。',
 '9 月 15 日 ， 台积电 、 高通 、 等 华为 的 重要 ， 只要 没有 美国 的 相关 许可证 ， 都 无法 供应 芯片 给 华为 而 中芯国际 等 国产 芯片 ， 也 因 采用 美国 技术 ， 而 无法 供货 给 华为 目前 华为 部分 的 手机 产品 货 少 的 现象 ， 若 该 形势 下去 ， 华为 手机 业务 遭

## 闭包数据增强

数据集中每条数据有两个句子 \
a, b, 1\
a, c, 1\
a, d, 0\
a~b, a~c => b~c\
a~b, ad不相似 => bd不相似

In [ ]:
import pandas as pd
from tqdm import tqdm
import json
def parse_data(path, data_type='train'):
    sentence_a = []
    sentence_b = []
    labels = []
    with open(path, 'r', encoding='utf8') as f:
        for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
            line = json.loads(line)
            sentence_a.append(line['sentence1'])
            sentence_b.append(line['sentence2'])
            if data_type != 'test':
                labels.append(int(line['label']))
            else:
                labels.append(0)
    df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns=['text_a', 'text_b', 'labels'])
    return df

In [ ]:
train_df = parse_data('data/data100821/train.json', data_type='train')

Reading train data: 100%|██████████| 34334/34334 [00:00<00:00, 203895.10it/s]


In [ ]:
train_df.head()

,text_a,text_b,labels
0,蚂蚁借呗等额还款可以换成先息后本吗,借呗有先息到期还本吗,0
1,蚂蚁花呗说我违约一次,蚂蚁花呗违约行为是什么,0
2,帮我看一下本月花呗账单有没有结清,下月花呗账单,0
3,蚂蚁借呗多长时间综合评估一次,借呗得评估多久,0
4,我的花呗账单是***，还款怎么是***,我的花呗，月结出来说让我还***元，我自己算了一下详细名单我应该还***元,1


In [51]:
for data in train_df.groupby(by=['text_a']):
    # print(type(data))
    # print(len(data))
    # print(data[0]) # text_a句子
    # print('-------')
    # print(data[1]) # 以text_a为主键返回的dataFrame
    if len(data[1])==2:
        print(data[0])
        print(data[1])
        print(data[1].info())
        break

为什么不能用蚂蚁借呗
           text_a           text_b  labels
3227   为什么不能用蚂蚁借呗           开通蚂蚁借呗       0
30332  为什么不能用蚂蚁借呗  为什么我的淘宝提示可以蚂蚁借呗       0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 3227 to 30332
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text_a  2 non-null      object
 1   text_b  2 non-null      object
 2   labels  2 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 64.0+ bytes
None


a : 为什么我开通不了花呗 b: 我一直想买苹果 p，没钱，想分期付款，除了花呗，还有什么可以  c:我开通不了蚂蚁花呗 
ab 不相似， ac相似，=》bc不相似


In [53]:
from collections import defaultdict
def aug_group_by_a(df):
    aug_data = defaultdict(list)
    # 以text_a中的句子为 a
    for g, data in df.groupby(by=['text_a']):
        if len(data)<2:
            continue
        for i in range(len(data)):
            for j in range(i+1, len(data)):
                # 取出b的值，a,b的label
                row_i_text = data.iloc[i, 1]
                row_i_label = data.iloc[i, 2]

                # 取出c的值，a,c的label
                row_j_text = data.iloc[j, 1]
                row_j_label = data.iloc[j, 2]

                if row_i_label == row_j_label == 0:
                    continue
                
                aug_label = 1 if row_i_label == row_j_label == 1 else 0 

                aug_data['text_a'].append(row_i_text)
                aug_data['text_b'].append(row_j_text)
                aug_data['labels'].append(aug_label)
    return pd.DataFrame(aug_data)

In [54]:
aug_train_a = aug_group_by_a(train_df)
print(aug_train_a)

                               text_a                   text_b  labels
0    我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以                我开通不了蚂蚁花呗       0
1    我一直想买苹果***p，没钱，想分期付款，除了花呗，还有什么可以  你直接帮我查一下 确实开通不了花呗吗？我的账户       0
2                          为什么不可以开通花呗                我开通不了蚂蚁花呗       0
3                          为什么不可以开通花呗  你直接帮我查一下 确实开通不了花呗吗？我的账户       0
4                            我为何打不开花呗                我开通不了蚂蚁花呗       0
..                                ...                      ...     ...
223                           你是人工服务吗                   需求人工客服       0
224                           你是人工服务吗                   联系人工客服       1
225                            需求人工客服                   联系人工客服       0
226                     身份证过期可以用蚂蚁借呗吗           身份证过期可以注册蚂蚁借呗吗       0
227                            人工关闭花呗     一个帐户的花呗关，另一个帐户的花呗怎么开       0

[228 rows x 3 columns]


In [55]:
from time import sleep
from tqdm import tqdm
for i in tqdm(range(60*15), desc='现在是休息时间，看录播的同学可以跳过哦～'):
    sleep(1)

现在是休息时间，看录播的同学可以跳过哦～: 100%|██████████| 900/900 [15:02<00:00,  1.00s/it]


## UDA（Unsupervised Data Augmentation for Consistency Training）用于一致性训练的无监督数据增强

 ![UDA1](https://img-blog.csdnimg.cn/c9cb603261a1497c8093ee669b7923f2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![UDA2](https://img-blog.csdnimg.cn/3e1a474a04564517bf8dafd3c017ec82.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

### 一致性训练
### 数据增强 是 创建 逼近 真实的训练数据，并且不改变其标签
### a, b 1  a c 1 -> b, c 1
### 增强前和增强后 标签 应该保持一致，利用这个特性训练

![UDA4](https://img-blog.csdnimg.cn/fa4442776d67471ab8e5c707d8ff6752.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

$q(\hat{x}|x)$ 根据原始example $x$ 绘制 $\hat{x}$ 并且要求$x$与$\hat{x}$标签相同


![UDA3](https://img-blog.csdnimg.cn/519f920759584cb6aeefce3d978974dd.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![unsupervised data augmentation](https://img-blog.csdnimg.cn/86f3aad1042b4bf381d662e9c3b48f0a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

 半监督学习利用无标签样本增强模型鲁棒性
 * 给定输入x, 计算输出分布$p_{\theta}(y|x)$, 同时，给输入x进行数据增强，计算出分布$p_{\theta}(y|\hat x)$.
 * 给两个分布计算KL散度

![UDA5](https://img-blog.csdnimg.cn/9d10da70d1d0467e93ef5bb1267ac87f.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![code](https://img-blog.csdnimg.cn/d97f35fd41e0485185f40d50f4fd8e8d.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [56]:
from bucket_sampler import SortedSampler, BucketBatchSampler
from EMA import *

In [57]:
import random
import numpy as np
import torch
config = {
        'train_file_path': 'data/data100821/train.json',
        'dev_file_path': 'data/data100821/dev.json',
        'test_file_path': 'data/data100821/test.json',
        'output_path': '.',
        'model_path': 'data/data94445',
        'batch_size': 16,
        'num_epochs': 1,
        'max_seq_len': 64,
        'learning_rate': 2e-5,
        'weight_decay': 0.01,
        'use_bucket': True,
        'bucket_multiplier': 200,
        'unsup_data_ratio': 1.5,
        'uda_softmax_temp': 0.4,
        'uda_confidence_threshold': 0.8,
        'device': 'cuda',
        'n_gpus': 0,
        'logging_step': 400,
        'ema_start_step': 500,
        'ema_start': False,
        'seed': 2021
    }

if not torch.cuda.is_available():
    config['device'] = 'cpu'
else:
    config['n_gpus'] = torch.cuda.device_count()
    config['batch_size'] *= config['n_gpus']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2021

In [58]:
! pip install transformers==4.0.1

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.4MB 15.2MB/s eta 0:00:01
     |████████████████████████████████| 40kB 19.8MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 14.6MB/s eta 0:00:01     |█████████████████▏              | 1.6MB 14.6MB/s eta 0:00:01


In [59]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(config['model_path'])

In [60]:
def build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append(inputs_dict['input_ids'])
    inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
    inputs['attention_mask'].append(inputs_dict['attention_mask'])
    inputs['labels'].append(label)

# 对偶数据增强
sentence1: a
sentence2: b
a,b, 1 变成 b,a 1

In [61]:
# 无监督BERT输入
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
                     
    rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
    inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'],rl_inputs_dict['token_type_ids']))
    inputs['attention_mask'].append((lr_inputs_dict['attention_mask'],rl_inputs_dict['attention_mask']))
    inputs['labels'].append(label)

In [63]:
def read_data(config, tokenizer):
    train_df = parse_data(config['train_file_path'], data_type='train')
    dev_df = parse_data(config['dev_file_path'], data_type='dev')
    test_df = parse_data(config['test_file_path'], data_type='test')

    data_df = {'train': train_df, 'dev': dev_df, 'test': test_df}
    processed_data = {}
    unsup_data = defaultdict(list)
    for data_type, df in data_df.items():
        inputs = defaultdict(list)
        if data_type == 'train':
            reversed_inputs = defaultdict(list)
        
        for i, row in tqdm(df.iterrows(), desc=f'Preprocessing {data_type} data', total=len(df)):
            label=0 if data_type == 'test' else row[2]
            sentence_a, sentence_b = row[0], row[1]
            build_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer)
            
            # 测试时刻增强
            if data_type.startswith('test'):
                build_bert_inputs(inputs, label, sentence_b, sentence_a, tokenizer)
            
            # train, dev, test
            build_unsup_bert_inputs(unsup_data, label, sentence_a, sentence_b, tokenizer)
        
        processed_data[data_type] = inputs
    
    processed_data['unsup_data'] = unsup_data
    return processed_data

# processed_data
# {
#    'train':,
#    'dev':,
#    'test':,
#    'unsup_data':   # 数据量最大的
# }

In [64]:
data = read_data(config, tokenizer)

Preprocessing test data: 100%|██████████| 3861/3861 [00:09<00:00, 413.86it/s]


In [65]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):
    def __init__(self, data_dict):
        super(AFQMCDataset, self).__init__()
        self.data_dict = data_dict
    
    def __getitem__(self, index):
        data = (self.data_dict['input_ids'][index], self.data_dict['token_type_ids'][index],
                self.data_dict['attention_mask'][index], self.data_dict['labels'][index])
        return data
    
    def __len__(self):
        return len(self.data_dict['input_ids'])

```
# 无监督BERT输入
def build_unsup_bert_inputs(inputs, label, sentence_a, sentence_b, tokenizer):
    lr_inputs_dict = tokenizer.encode_plus(sentence_a, sentence_b, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
                     
    rl_inputs_dict = tokenizer.encode_plus(sentence_b, sentence_a, add_special_tokens=True,
                          return_token_type_ids=True, return_attention_mask=True)
    
    inputs['input_ids'].append((lr_inputs_dict['input_ids'], rl_inputs_dict['input_ids']))
    inputs['token_type_ids'].append((lr_inputs_dict['token_type_ids'],rl_inputs_dict['token_type_ids']))
    inputs['attention_mask'].append((lr_inputs_dict['attention_mask'],rl_inputs_dict['attention_mask']))
    inputs['labels'].append(label)
```

In [66]:
class Collator:
    def __init__(self, max_seq_len, tokenizer):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
    
    def pad_and_truncate(self, input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len):
        input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype=torch.long)
        token_type_ids = torch.zeros_like(input_ids)
        attention_mask = torch.zeros_like(input_ids)
        for i in range(len(input_ids_list)):
            seq_len = len(input_ids_list[i])
            if seq_len<=max_seq_len:
                input_ids[i,:seq_len] = torch.tensor(input_ids_list[i], dtype=torch.long)
                token_type_ids[i,:seq_len] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
                attention_mask[i,:seq_len] = torch.tensor(attention_mask_list[i], dtype=torch.long)
            
            else:
                input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len-1]+[self.tokenizer.sep_token_id], dtype=torch.long)
                token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype=torch.long)
                attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype=torch.long)
            
        labels = torch.tensor(labels_list, dtype=torch.long)
        return input_ids, token_type_ids, attention_mask, labels

    def __call__(self, examples):
        input_ids_list, token_type_ids_list, attention_mask_list, labels_list= list(zip(*examples))
        cur_max_seq_len = max(len(input_ids) for input_ids in input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)

        input_ids, token_type_ids, attention_mask, labels = self.pad_and_truncate(input_ids_list, token_type_ids_list, attention_mask_list, labels_list, max_seq_len)
        data_dict = {
            'input_ids': input_ids,
            'token_type_ids': token_type_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
        return data_dict

In [67]:
collate_fn = Collator(config['max_seq_len'], tokenizer)

In [69]:
# UDA
class UnsupAFQMCDataset(Dataset):
    def __init__(self, data_dict):
        super(UnsupAFQMCDataset, self).__init__()
        self.data_dict = data_dict
    
    def __getitem__(self, index):
        input_ids = self.data_dict['input_ids'][index]
        token_type_ids = self.data_dict['token_type_ids'][index]
        attention_mask = self.data_dict['attention_mask'][index]
        labels = self.data_dict['labels'][index]
        return (input_ids[0], token_type_ids[0], attention_mask[0],
                input_ids[1], token_type_ids[1], attention_mask[1],
                labels)

    def __len__(self):
        return len(self.data_dict['input_ids'])

In [72]:
class UnsupCollator(Collator):
    def __init__(self, max_seq_len, tokenizer):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
  

    def __call__(self, examples):
        (ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list,
        ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list) = list(zip(*examples))

        cur_max_seq_len = max(len(input_ids) for input_ids in ab_input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)

        # 分批整ab, ba
        ab_input_ids, ab_token_type_ids, ab_attention_mask, labels = self.pad_and_truncate(ab_input_ids_list, ab_token_type_ids_list, ab_attention_mask_list, labels_list, max_seq_len)
        ba_input_ids, ba_token_type_ids, ba_attention_mask, labels = self.pad_and_truncate(ba_input_ids_list, ba_token_type_ids_list, ba_attention_mask_list, labels_list, max_seq_len)

        data_dict = {
            'ab_input_ids': ab_input_ids,
            'ab_token_type_ids': ab_token_type_ids,
            'ab_attention_mask': ab_attention_mask,
            'ba_input_ids': ba_input_ids,
            'ba_token_type_ids': ba_token_type_ids,
            'ba_attention_mask': ba_attention_mask,
            'labels': labels

        }
        return data_dict

In [73]:
from torch.utils.data import DataLoader, RandomSampler
def build_dataloader(config, data, tokenizer):
    train_dataset = AFQMCDataset(data['train'])
    dev_dataset = AFQMCDataset(data['dev'])
    test_dataset = AFQMCDataset(data['test'])

    unsup_dataset = UnsupAFQMCDataset(data['unsup_data'])

    collate_fn = Collator(config['max_seq_len'], tokenizer)
    unsup_collate_fn = UnsupCollator(config['max_seq_len'], tokenizer)

    if config['use_bucket']:
        # 监督数据
        train_sampler = RandomSampler(train_dataset)
        bucket_sampler = BucketBatchSampler(train_sampler, batch_size=config['batch_size'],
                                           drop_last=False, sort_key=lambda x:len(train_dataset[x][0]),
                                           bucket_size_multiplier=config['bucket_multiplier'])

        train_dataloder = DataLoader(dataset=train_dataset, batch_sampler=bucket_sampler, num_workers=4, collate_fn=collate_fn)

        # 无监督数据
        unsup_sampler = RandomSampler(unsup_dataset)
        unsup_bucket_sampler = BucketBatchSampler(unsup_sampler, batch_size=int(config['batch_size']*config['unsup_data_ratio']),
                                           drop_last=False, sort_key=lambda x:len(train_dataset[x][0]),
                                           bucket_size_multiplier=config['bucket_multiplier'])
        unsup_dataloader = DataLoader(dataset=unsup_dataset, batch_sampler=unsup_bucket_sampler, num_workers=4, collate_fn=unsup_collate_fn)
    
    else:
        # 监督数据
        train_dataloder = DataLoader(dataset=train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=4, collate_fn=collate_fn)

        # 无监督数据
        unsup_dataloader = DataLoader(dataset=unsup_dataset, batch_size=int(config['batch_size']*config['unsup_data_ratio']), shuffle=True,
                                      num_workers=4, collate_fn=unsup_collate_fn)
    
    dev_dataloader = DataLoader(dataset=dev_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, collate_fn=collate_fn)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=4, collate_fn=collate_fn)

    return unsup_dataloader, train_dataloder, dev_dataloader, test_dataloader

In [74]:
unsup_dataloader, train_dataloder, dev_dataloader, test_dataloader = build_dataloader(config, data, tokenizer)

In [75]:
for i in train_dataloder:
    print(i)
    break

{'input_ids': tensor([[ 101, 6010, 6009,  955, 1446, 5543, 5314, 2769,  115,  115,  115,  674,
         4638, 7583, 2428,  720, 8024, 6468, 6468,  102, 6010, 6009,  955, 1446,
         6820,  749, 3621, 3300, 7583, 2428, 1408,  102],
        [ 101, 2582, 3416, 1377,  809, 2990, 1184, 6820, 3378,  671, 3309, 4638,
         5709, 1446,  102, 5709, 1446, 1146, 3309,  749,  809, 1400, 8024, 2990,
         1184, 6820, 3621, 1377,  809, 1048, 1343,  102],
        [ 101, 2769, 4638, 5709, 1446, 3300, 3867, 6589, 8024, 2769, 2682, 2990,
         1184, 6820, 3926,  102, 5709, 1446, 2990, 1184, 6820, 3621, 8024, 6821,
          702, 1216, 5543, 3766, 3300, 2823, 1168,  102],
        [ 101, 2769, 6821, 3221, 5709, 1446, 4638, 3297,  856, 6820, 3621, 6848,
         7555, 8024, 6820,  749,  115,  115,  115,  102, 5709, 1446, 8024, 3297,
          856, 6820, 3621,  671, 4684, 1927, 6571,  102],
        [ 101, 3118,  802, 2140, 1384, 4772, 2940,  749, 8024, 5709, 1446, 6820,
         3221, 1333, 3341

In [76]:
from sklearn import metrics
def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
            batch_cuda['mode'] = 'val'
            loss, logits = model(**batch_cuda)[:2]

            if config['n_gpus'] > 1:
                loss = loss.mean()

            val_loss += loss.item()
            preds.append(logits.argmax(dim=-1).detach().cpu())

    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    f1 = metrics.f1_score(labels, preds)
    acc = metrics.accuracy_score(labels, preds)
    return avg_val_loss, f1, acc

![BertForSequenceClassification](https://img-blog.csdnimg.cn/65a419a6c58a4b07883a3c91084e7cde.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![BertForSequenceClassification1](https://img-blog.csdnimg.cn/0c2a5f0612aa4e42a1dfdf0e0352e0f2.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [78]:
from transformers import BertForSequenceClassification
import torch.nn as nn
class BertForAFQMC(BertForSequenceClassification):
    def forward(self, input_ids, token_type_ids, atttention_mask, labels=None, mode='train'):
        outputs = self.bert(input_ids, token_type_ids=token_type_ids, atttention_mask=atttention_mask, output_hidden_states=True)
        
        # [batch_size, hidden_size]
        pooled_output = outputs[1]
        pooled_output = self.drop(pooled_output)

        logits = self.classifier(pooled_output)
        outputs = (logits, )

        if mode == 'val':
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels.view(-1))

            outputs = (loss, ) +outputs
        return outputs

![addtitional Training1](https://img-blog.csdnimg.cn/5916fe8ae028469bb877d15a1ac566de.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![UDA5](https://img-blog.csdnimg.cn/1ddf28077b88449aa84e0391149467e4.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
def get_tsa_threshold(total_steps, global_steps):
    return np.exp((global_steps / total_steps - 1) * 5) / 2 + 0.5

$$CE = -plogq$$   
$$E = -plogp$$
p-真实分布（已知） q-预测分布
$$KLDiv(p||q) = \sum_{i=1}^{N}p(x_{i})(logp(x_{i})- logq(x_{i}))$$

![cross entropy](https://img-blog.csdnimg.cn/4572c78d76624c49b01b96a1cba42279.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![KL](https://img-blog.csdnimg.cn/189e4bc953904c199afbc7e6a11e5d9a.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
from transformers import AdamW
from tqdm import trange
import os
def train(config, train_dataloader, dev_dataloader, unsup_dataloader=None):
    model = BertForAFQMC.from_pretrained(config['model_path'])

    optimizer = AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])
    model.to(config['device'])
    total_steps = len(unsup_dataloader) * config['num_epochs']
    epoch_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.
    best_acc = 0.
    best_model_path = ''

    if config['n_gpus'] > 1:
        model = nn.DataParallel(model)

    train_iterator = iter(train_dataloader)
    for _ in epoch_iterator:
        unsup_iterator = tqdm(unsup_dataloader, desc='Training', total=len(unsup_dataloader))
        model.train()
        for unsup_batch in unsup_iterator:
            cur_bs = unsup_batch['ab_input_ids'].size(0)
            try:
                sup_batch = next(train_iterator)
            except StopIteration:
                train_iterator = iter(train_dataloader)
                sup_batch = next(train_iterator)

            grad_data, no_grad_data = get_data(sup_batch, unsup_batch, config)

            unsup_loss_mask, unsup_probs = forward_no_grad(no_grad_data, config, model)

            loss, tsa_threshold, unsup_loss, sup_loss = forward_with_grad(
                unsup_loss_mask, unsup_probs, config, cur_bs, model, grad_data, total_steps, global_steps
            )
            

            model.zero_grad()
            loss.backward()
            
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            if config['ema_start']:
                ema.update()

            train_loss += loss.item()
            global_steps += 1

            unsup_iterator.set_postfix_str(f'running training loss: {loss.item():.4f}')
        
            if global_steps % config['logging_step'] == 0:
                if global_steps >= config['ema_start_step'] and not config['ema_start']:
                    print('\n>>> EMA starting ...')
                    config['ema_start'] = True
                    ema = EMA(model.module if hasattr(model, 'module') else model, decay=0.999)

                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                logging_loss = train_loss

                if config['ema_start']:
                    ema.apply_shadow()
                val_loss, f1, acc = evaluation(config, model, dev_dataloader)

                print_log = f'\n>>> training loss: {print_train_loss:.6f}, valid loss: {val_loss:.6f}, '

                if acc > best_acc:
                    model_save_path = os.path.join(config['output_path'],
                                                   f'checkpoint-{global_steps}-{acc:.6f}')
                    model_to_save = model.module if hasattr(model, 'module') else model
                    model_to_save.save_pretrained(model_save_path)
                    best_acc = acc
                    best_model_path = model_save_path

                print_log += f'valid f1: {f1:.6f}, valid acc: {acc:.6f}'

                print(print_log)
                model.train()
                if config['ema_start']:
                    ema.restore()

    return model, best_model_path

In [ ]:
model, best_model_path = train(config, train_dataloader, dev_dataloader, unsup_dataloader)

In [ ]:
from zipfile import ZipFile
def predict(config, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Predicting', total=len(test_dataloader))
    test_preds = []

    model.eval()
    with torch.no_grad():
        for batch in test_iterator:
            batch_cuda = {item: value.to(config['device']) for item, value in list(batch.items())}
            logits = model(**batch_cuda)[0]
            probs = torch.softmax(logits, dim=-1)
            test_preds.append(probs[:, 1].detach().cpu())
    

    test_preds = torch.cat(test_preds)
    test_preds = torch.stack(test_preds.split(2), dim=0).mean(dim=1).numpy()
    submission_path = os.path.join(config['output_path'], 'submission.tsv')
    test_df = pd.DataFrame(data={'prediction': test_preds})
    test_df.to_csv(submission_path, index=False, header=False, encoding='utf8', sep='\t')
    with ZipFile(os.path.join(config['output_path'], 'submission.zip'), 'w') as myzip:
        myzip.write(submission_path, 'submission.tsv')

In [ ]:
predict(config, model, test_dataloader)